# Mini-projet COCOMA - allocation de resssources

Le mini-projet consiste simplement à produire un notebook qui permettra d'illustrer deux protocoles pour la coordination à base d'enchères:

- les protocoles SSI (version classique et regret)

- le protocole CBAA 

On s'nspirera de la fiche exemple donnée en pdf. Pour les deux protocoles, les agents/robots doivent être situés sur un graphe, dont les arcs sont valués (distance).

In [1]:
from problem import Problem
#import utils
import numpy as np
import sys
import importlib
#importlib.reload(utils)

## 1. Les protocoles SSI

### 1.1. Version classique

In [2]:
agents=["r1","r2"]
resources=["o1","o2","o3","o4"]

grid=np.empty([8, 8], dtype = object)

grid[2][2]="o2"
grid[2][4]="o4"
grid[4][4]="r2"
grid[5][5]="o1"
grid[5][2]="r1"
grid[7][4]="o3"
grid


array([[None, None, None, None, None, None, None, None],
       [None, None, None, None, None, None, None, None],
       [None, None, 'o2', None, 'o4', None, None, None],
       [None, None, None, None, None, None, None, None],
       [None, None, None, None, 'r2', None, None, None],
       [None, None, 'r1', None, None, 'o1', None, None],
       [None, None, None, None, None, None, None, None],
       [None, None, None, None, 'o3', None, None, None]], dtype=object)

In [107]:
sii(grid,agents,resources)

Round:  0
	 Resource:  o1  ==> Agent:  r2 

Round:  1
	 Resource:  o2  ==> Agent:  r1 

Round:  2
	 Resource:  o3  ==> Agent:  r2 

Round:  3
	 Resource:  o4  ==> Agent:  r1 

----------- Solution ------------
Agent:  r1  :  ['o2', 'o4']
Agent:  r2  :  ['o1', 'o3']


In [122]:
ssi_regret(grid,agents,resources)

Round:  0
	 Available resources:  ['o1', 'o2', 'o3', 'o4']
	 Resource:  o4  ==> Agent:  r2 



Round:  1
	 Available resources:  ['o1', 'o2', 'o3']
	 Resource:  o1  ==> Agent:  r1 



Round:  2
	 Available resources:  ['o2', 'o3']
	 Resource:  o2  ==> Agent:  r2 



Round:  3
	 Available resources:  ['o3']
	 Resource:  o3  ==> Agent:  r1 



----------- Solution ------------
Agent:  r1  :  ['o1', 'o3']
Agent:  r2  :  ['o4', 'o2']


In [121]:
def my_bids(grid,resources,i,agent,nodes_agent):
    bids=[]
    b=[]
    for r in resources:
        #calcul du cout
        g=gain(grid,r,i,agent,nodes_agent)
        bids.append(g)
        b.append(g[0])
    return bids,b
        
def gain(grid,r,i,agent,nodes_agent):
    cp = 0
    pos_a_x=np.where(grid == agent)[0][0] 
    pos_a_y=np.where(grid == agent)[1][0]
    pos_r_x=np.where(grid == r)[0][0]
    pos_r_y=np.where(grid == r)[1][0]
    min_cost=sys.maxsize
    cp_min=-1
    
    if(len(nodes_agent)==0):
        return (distance(pos_a_x,pos_r_x,pos_a_y,pos_r_y),cp)
    while(cp<len(nodes_agent)+1):
        if(cp==0):
            pos_node1_x=np.where(grid == nodes_agent[cp])[0][0]
            pos_node1_y=np.where(grid == nodes_agent[cp])[1][0]
            d=distance(pos_a_x,pos_r_x,pos_a_y,pos_r_y)+distance(pos_node1_x,pos_r_x,pos_node1_y,pos_r_y)-distance(pos_node1_x,pos_a_x,pos_node1_y,pos_a_y)
            if (d<min_cost):
                cp_min=cp
                min_cost=d
        elif(cp==len(nodes_agent)):
            pos_node1_x=np.where(grid == nodes_agent[cp-1])[0][0]
            pos_node1_y=np.where(grid == nodes_agent[cp-1])[1][0]
            d=distance(pos_node1_x,pos_r_x,pos_node1_y,pos_r_y)
            if(d<min_cost):
                cp_min=cp
                min_cost=d
        else:
            pos_node1_x=np.where(grid == nodes_agent[cp-1])[0][0]
            pos_node1_y=np.where(grid == nodes_agent[cp-1])[1][0]
            pos_node2_x=np.where(grid == nodes_agent[cp])[0][0]
            pos_node2_y=np.where(grid == nodes_agent[cp])[1][0]
            d=distance(pos_node1_x,pos_r_x,pos_node1_y,pos_r_y)+distance(pos_node2_x,pos_r_x,pos_node2_y,pos_r_y)-distance(pos_node1_x,pos_node2_x,pos_node1_y,pos_node2_y)
            if(d<min_cost):
                cp_min=cp
                min_cost=d
        cp+=1
    return (min_cost,cp)
        
    
def ssi_regret(grid,agents,resources):
    available_resources=resources.copy()    
    nodes_agents=initbags(len(agents))
    round_alloc=0
    while(len(available_resources)!=0):
        print("Round: ",round_alloc)
        
        print("\t Available resources: ",available_resources)
        i=0
        regrets=[]
        tmp_bids=[]
        costs=[]
        for agent in agents:
            #calcul
            bids_cp,evaluation=my_bids(grid,available_resources,i,agent,nodes_agents[i])
            costs.append(evaluation)
            tmp_bids.append(bids_cp)
            i+=1
        #regret 
        costs=np.array(costs)
        k=0
        while(k<len(costs[0])):
            col=np.array(costs)[:,k]
            #print("col ",col)
            tmp_c=np.flip(np.sort(col))
            first_max=tmp_c[0]
            second_max=tmp_c[1]
            regret=abs(second_max-first_max)
            regrets.append(regret)
            k+=1
        top_regret=max(regrets)
        
        index_col_max_regret=np.where(regrets==top_regret)[0][0]
        
        col=costs[:,index_col_max_regret]
        #print("index_col_max_regret ",index_col_max_regret)
        #print("costs ",costs)
        #print("regrets ",regrets)
        #print("col ",col)
        min_bid=min(col)
        index_min_bid=np.where(col==min_bid)[0][0]
        agent_winner=agents[index_min_bid]
        res_won=available_resources[index_col_max_regret]
        print("\t Resource: ",res_won," ==> Agent: ",agent_winner,"\n")
        available_resources.remove(res_won)
        #insert the new node
        cp_insertion=tmp_bids[index_min_bid][index_col_max_regret][1]
        nodes_agents[index_min_bid].insert(cp_insertion,res_won)
        #print("Agent's sequences ",nodes_agents)
        print("\n")
        round_alloc+=1
        
    i=0
    print("----------- Solution ------------")
    for agent in agents:
        print("Agent: ",agent," : ",nodes_agents[i])
        i+=1

In [106]:
def distance(x1,x2,y1,y2):
    return abs(x1-x2) + abs(y1-y2)


def initbags(n):
    m=[]
    for i in range (n):
        m.append([])
    return m


def bid(grid,r,i,agent,positions_agents):
      pos_r_x=np.where(grid == r)[0][0]
      pos_r_y=np.where(grid == r)[1][0]
      #init the nearest target with the robot's position
      nearest_pos_target_x=positions_agents[i][0][0]
      nearest_pos_target_y=positions_agents[i][0][1]
      distance_nearest_target=distance(nearest_pos_target_x,pos_r_x,nearest_pos_target_y,pos_r_y)

      for j in range(1,len(positions_agents[i])):
          pos_a_x=positions_agents[i][j][0]
          pos_a_y=positions_agents[i][j][1]
          d=distance(pos_a_x,pos_r_x,pos_a_y,pos_r_y)
          if(d<distance_nearest_target):
              nearest_pos_target_x=pos_a_x
              nearest_pos_target_y=pos_a_y
              distance_nearest_target=distance(nearest_pos_target_x,pos_r_x,nearest_pos_target_y,pos_r_y)
      return [distance_nearest_target,i]
         
def sii(grid,agents,resources):
    bags=initbags(len(agents))
    positions_agents=initbags(len(agents))
    
    cp=0
    #init position of agents
    for a in agents:
        pos_a_x=np.where(grid == a)[0][0] 
        pos_a_y=np.where(grid == a)[1][0]
        positions_agents[cp].append([pos_a_x,pos_a_y])
        cp+=1
        
    cp=0
    for r in resources:
         print("Round: ",cp)
         i=0
         bids=[]
         for agent in agents:
             bids.append(bid(grid,r,i,agents[i],positions_agents))
             i+=1
         #print(bids)
         bids=np.array(bids)
         min_bid=min(bids[:,0])
         index_winner=np.where(bids[:,0] == min_bid)[0][0]
         agent_winner=agents[index_winner]
         #print("agent_winner ",agent_winner)
         pos_r_x=np.where(grid == r)[0][0]
         pos_r_y=np.where(grid == r)[1][0]
         positions_agents[index_winner].append([pos_r_x,pos_r_y])
         #print("positions_agents ",positions_agents)
         bags[index_winner].append(r)
         print("\t Resource: ",r," ==> Agent: ",agent_winner,"\n")
         cp+=1
    i=0
    print("----------- Solution ------------")
    for b in bags:
        print("Agent: ",agents[i]," : ",b)
        i+=1           

### 1.2. Version regret

## 2. Le protocoles CBAA